In [1]:
########################################################################################################################
# Gurobi_All_in_One_Jupyter_GUI.ipynb
# Version: V1.0 | Last Updated: 20230829
# This model draws inspiration from and is built upon the foundation set by "Excel_to_Gurobi_JUP.ipynb" by Zack Zhu
# Please read "Handbook of Gurobi All_in_One Jupyter GUI" prior to usage.
# Should you face any challenges or issues, please don't hesitate to reach out to me (Henry Wu):
# ihenrywu.ca@gmail.com or https://www.linkedin.com/in/ihenrywu/
# Enjoy the Optimal Decision Making!
########################################################################################################################


from gurobipy import *
import pandas as pd
import numpy as np

import qgrid
import ipywidgets as widgets
from IPython.display import display, clear_output, HTML, Javascript



# Checks if the Gurobi license is valid
def show_warning_message(message):
    js_code = f'alert("{message}")'
    display(Javascript(js_code))
    
def check_gurobi_license():
    try:
        # Create a temporary model to check the license
        temp_model = Model("temp_model")
        # If no exception is raised, the license is valid
        pass
    except GurobiError as e:
        if 'License expired' in str(e):
            show_warning_message("Warning: Your Gurobi license has expired!")
        else:
            print(f"An error occurred: {e}")
        return False    
    
check_gurobi_license()


# Display Category options
radio_buttons = widgets.RadioButtons(
    options=['General Model',
             'Transportation Problem', 
             'Transshipment Problem', 
             'Assignment Problem', 
             'Shortest Path Problem', 
             'Maximum Flow Problem', 
             'Traveling Salesperson Problem'
             ],
    description='Category:',
    disabled=False
)

radio_buttons.index = None
output = widgets.Output()


# Display Dropdown choices
objective_dropdown = widgets.Dropdown(options=['min', 'max'], description='Objective:')
variable_type_dropdown = widgets.Dropdown(options=['cont', 'bin', 'int'], description='Variable:')
range_dropdown = widgets.Dropdown(options=['non-negative', 'all range'], description='Range:')
problem_type_dropdown = widgets.Dropdown(options=['MCNF', 'MFP', 'TSP', 'General'], description='Type:')


def update_defaults(change):
    """
    Update default values of dropdowns based on the selected problem category.
    """
    selected_option = change['new']
    
    if selected_option == 'Maximum Flow Problem':
        problem_type_dropdown.value = 'MFP'
    elif selected_option == 'Traveling Salesperson Problem':
        variable_type_dropdown.value = 'int'
        problem_type_dropdown.value = 'TSP'
    elif selected_option == 'General Model':
        range_dropdown.value = 'all range'
        problem_type_dropdown.value = 'General'
    else:
        # Handle unexpected selected options if needed
        pass

def on_radio_button_change(change):
    with output:
        clear_output(wait=True)
        display(widgets.HBox([objective_dropdown, variable_type_dropdown, range_dropdown, problem_type_dropdown]), first_next_button)

# Link the update function to the radio_buttons
radio_buttons.observe(update_defaults, names='value')
radio_buttons.observe(on_radio_button_change, names='value')


# Input boxes
style = {'description_width': '100px'}
box_count_supply_nodes = widgets.BoundedIntText(value=0, min=0, max=1000, description='Supply Node:', disabled=False, style=style)
box_count_demand_nodes = widgets.BoundedIntText(value=0, min=0, max=1000, description='Demand Node:', disabled=False, style=style)
box_count_transfer_nodes = widgets.BoundedIntText(value=0, min=0, max=1000, description='Transfer Node:', disabled=False, style=style)
box_count_start_node = widgets.BoundedIntText(value=1, description='Start Node:', disabled=True, style=style)
box_count_end_node = widgets.BoundedIntText(value=1, description='End Node:', disabled=True, style=style)


# Store qgrid widgets globally for later access
qgrid_supply_widget = None
qgrid_demand_widget = None
qgrid_transfer_widget = None
qgrid_start_widget = None
qgrid_end_widget = None
qgrid_matrix_widget = None


# Functions
def on_first_next_button_click(button):
    """
    Handles the action when the first 'Next' button is clicked.
    Depending on the selected radio button value, it displays the appropriate input widgets.

    Parameters:
    - button: The button widget instance.
    """
    #global box_count_supply_nodes, box_count_demand_nodes, box_count_transfer_nodes, box_count_start_node, box_count_end_node
    try:
        with output:
            # Decision based on radio button value
            if radio_buttons.value == 'General Model':
                general_model_set_variables()
            else:
                network_model_set_node_count()
        button.disabled = True
    except Exception as e:
        with output:
            display(HTML(f'<span style="color: red;">An error occurred: {e}</span>'))

def general_model_set_variables():
    """
    Displays the widgets for STEP 2 when 'General Model' is selected.
    """
    global qgrid_variables_widget
    
    display(HTML("<hr>"))
    display(HTML('<h3 style="background-color: yellow;">STEP 2: Set Variables</h3>'))
    
    # Display variable grid
    df_variables = pd.DataFrame({
        'Variable Name': ['' for _ in range(3)],
        'Objective coefficient': [0.0 for _ in range(3)],
        'Variable Lower Bound': [None for _ in range(3)],
        'Variable Upper Bound': [None for _ in range(3)]
    }, index=list(range(1, 4)))

    qgrid_variables_widget = qgrid.show_grid(df_variables, show_toolbar=True, grid_options={'forceFitColumns': False, 'defaultColumnWidth': 180, 'filterable': False})

    display(qgrid_variables_widget, input_constraints_button)

    
def network_model_set_node_count():
    """
    Displays the widgets for STEP 2 when 'General Model' is not selected. 
    """
    global box_count_supply_nodes, box_count_demand_nodes

    
    display(HTML("<hr>"))
    display(HTML('<h3 style="background-color: yellow;">STEP 2: Set Node Count</h3>'))

    # Check which option is selected in radio_buttons and display accordingly
    if radio_buttons.value in ['Transportation Problem', 'Assignment Problem']:
        display(widgets.VBox([box_count_supply_nodes, box_count_demand_nodes]), generate_tables_button) 
    elif radio_buttons.value == 'Traveling Salesperson Problem':
        display(widgets.VBox([box_count_transfer_nodes]), generate_tables_button)      
    elif radio_buttons.value == 'Maximum Flow Problem':
        display(widgets.VBox([box_count_start_node, box_count_end_node, box_count_transfer_nodes]), generate_tables_button)  
    elif radio_buttons.value == 'Shortest Path Problem':
        box_count_supply_nodes = widgets.BoundedIntText(value=1, min=0, description='Start Node:', disabled=True, style={'description_width': '100px'})
        box_count_demand_nodes = widgets.BoundedIntText(value=1, min=0, description='End Node:', disabled=True, style={'description_width': '100px'})
        display(widgets.VBox([box_count_supply_nodes, box_count_demand_nodes, box_count_transfer_nodes]), generate_tables_button)
    else:
        display(widgets.VBox([box_count_supply_nodes, box_count_demand_nodes, box_count_transfer_nodes]), generate_tables_button)

        
def on_input_constraints_button_click(button):
    """
    Handles the button click event for inputting constraints.
    Displays the constraints input grid to the user.
    """
    global qgrid_variables_widget, qgrid_constraints_widget, variable_name
    
    with output:
        display(HTML("<hr>"))
        display(HTML('<h3 style="background-color: yellow;">STEP 3: Set Constraints Manually</h3>'))
        display(HTML('<p>STEP 3: Constraints of Lower Bound, Upper Bound, and Non-Negative will be generated automatically.</p>'))

        # Fetch variable names and filter out empty ones
        variable_df = qgrid_variables_widget.get_changed_df()
        variable_df = variable_df[variable_df['Variable Name'] != '']

        variable_name = variable_df['Variable Name'].dropna().tolist()

        
        if not variable_name:
            print("Error: No variable names provided.")
            return
        
        # Create constraints DataFrame
        columns_name = variable_name + ['constraint type', 'RHS values']
        df_constraints = pd.DataFrame(0.0, columns=columns_name, index=range(1, 4))
        df_constraints['constraint type'] = ''
        
        # Display the constraints input grid
        qgrid_constraints_widget = qgrid.show_grid(df_constraints, show_toolbar=True, grid_options={'forceFitColumns': False, 'defaultColumnWidth': 100, 'filterable': False})
        display(qgrid_constraints_widget, generate_all_constraints_button)
     
    button.disabled = True
    
    
# create tables based on the problem type and number of nodes
def on_generate_tables_button_click(button):
    """
    Handles the button click event for generating tables.
    Displays the tables for node names and capacities.
    """
    global qgrid_supply_widget, qgrid_demand_widget, qgrid_transfer_widget, qgrid_start_widget, qgrid_end_widget, qgrid_matrix_widget
    global count_supply_nodes, count_demand_nodes, count_transfer_nodes

        
    with output:
        display(HTML("<hr>"))
        display(HTML('<h3 style="background-color: yellow;">STEP 3: Set Node Names & Capacities</h3>'))

        # Extract values from input boxes
        count_supply_nodes = box_count_supply_nodes.value
        count_demand_nodes = box_count_demand_nodes.value
        count_transfer_nodes = box_count_transfer_nodes.value
        
        # Create DataFrames based on input values
        df_supply = pd.DataFrame({
            'Name of Node': ['' for _ in range(count_supply_nodes)],
            'Capacity of Node': [0.0 for _ in range(count_supply_nodes)]
        }, index=list(range(1, count_supply_nodes + 1)))
        
        df_demand = pd.DataFrame({
            'Name of Node': ['' for _ in range(count_demand_nodes)],
            'Demand of Node': [0.0 for _ in range(count_demand_nodes)]
        }, index=list(range(1, count_demand_nodes + 1)))
        
        df_transfer = pd.DataFrame({
            'Name of Node': ['' for _ in range(count_transfer_nodes)]
        }, index=list(range(1, count_transfer_nodes + 1)))
              

        if radio_buttons.value in ['Assignment Problem', 'Shortest Path Problem']:
            df_supply['Capacity of Node'] = [1]* len(df_supply)
            df_demand['Demand of Node'] = [1]* len(df_demand)
            
            
        if radio_buttons.value == 'Maximum Flow Problem':
            df_start = pd.DataFrame({
                'Name of Node': ['' for _ in range(1)]
            }, index=list(range(1, 2)))

            df_end = pd.DataFrame({
                'Name of Node': ['' for _ in range(1)]
            }, index=list(range(1, 2)))              
        else:
            df_start = pd.DataFrame({
                'Name of Node': ['' for _ in range(0)]
            }, index=list(range(1, 1)))

            df_end = pd.DataFrame({
                'Name of Node': ['' for _ in range(0)]
            }, index=list(range(1, 1)))

            
        # Create qgrid widgets without the toolbar and store them
        qgrid_supply_widget = qgrid.show_grid(df_supply, show_toolbar=False, grid_options={'forceFitColumns': False, 'defaultColumnWidth': 200, 'filterable': False})
        qgrid_demand_widget = qgrid.show_grid(df_demand, show_toolbar=False, grid_options={'forceFitColumns': False, 'defaultColumnWidth': 200, 'filterable': False})
        qgrid_transfer_widget = qgrid.show_grid(df_transfer, show_toolbar=False, grid_options={'forceFitColumns': False, 'defaultColumnWidth': 200, 'filterable': False})
        qgrid_start_widget = qgrid.show_grid(df_start, show_toolbar=False, grid_options={'forceFitColumns': False, 'defaultColumnWidth': 200, 'filterable': False})
        qgrid_end_widget = qgrid.show_grid(df_end, show_toolbar=False, grid_options={'forceFitColumns': False, 'defaultColumnWidth': 200, 'filterable': False})

        # Display the qgrid widgets and the Generate Matrix button            
        if radio_buttons.value == 'Shortest Path Problem':
            display(HTML("<p>Start Node:</p>"))
            display(qgrid_supply_widget)
            display(HTML("<p>End Node:</p>"))
            display(qgrid_demand_widget)
        elif radio_buttons.value == 'Maximum Flow Problem':
            display(HTML("<p>Start Node:</p>"))
            display(qgrid_start_widget)
            display(HTML("<p>End Node:</p>"))
            display(qgrid_end_widget)
        else:
            if len(df_supply) > 0:
                display(HTML("<p>Supply Node:</p>"))
                display(qgrid_supply_widget)
            if len(df_demand) > 0 :
                display(HTML("<p>Demand Node:</p>"))
                display(qgrid_demand_widget)            

        if len(df_transfer) > 0:
            display(HTML("<p>Transfer Node:</p>"))
            display(qgrid_transfer_widget)
            
        display(generate_matrix_button)
        
    button.disabled = True


# create matrix based on the input of nodes
def fetch_names_from_widget(widget):
    """
    Fetch names from the provided widget.
    If the widget is None, return an empty list.
    """
    if widget:
        return widget.get_changed_df()['Name of Node'].values.tolist()
    return []

def on_generate_matrix_button_click(button):
    """
    Handles the button click event for generating matrices.
    Displays the flow amounts between nodes in matrix format.
    """
    global qgrid_supply_widget, qgrid_demand_widget, qgrid_transfer_widget, qgrid_start_widget, qgrid_end_widget, qgrid_matrix_widget
    global supply_names, demand_names, transfer_names, start_name, end_name
    
    with output:
        display(HTML("<hr>"))
        display(HTML('<h3 style="background-color: yellow;">STEP 4: Set Flow Amounts Between Nodes</h3>'))

        # Fetch names from widgets
        supply_names = fetch_names_from_widget(qgrid_supply_widget)
        demand_names = fetch_names_from_widget(qgrid_demand_widget)
        transfer_names = fetch_names_from_widget(qgrid_transfer_widget)
        start_name = fetch_names_from_widget(qgrid_start_widget)
        end_name = fetch_names_from_widget(qgrid_end_widget)

        # Determine row and column names for the matrix based on the selected problem type
        if radio_buttons.value == 'Maximum Flow Problem':
            row_names = supply_names + start_name + transfer_names + end_name
            column_names = transfer_names + demand_names + end_name + start_name
        else:
            row_names = supply_names + transfer_names
            column_names = transfer_names + demand_names
        
        # Create and display the matrix
        matrix_df = pd.DataFrame(0, index=row_names, columns=column_names)
        qgrid_matrix_widget = qgrid.show_grid(matrix_df, show_toolbar=False, grid_options={'forceFitColumns': False, 'defaultColumnWidth': 100, 'filterable': False})
        
        display(qgrid_matrix_widget, submit_button)
    
    button.disabled = True
    
    
def check_dataframe(df, index_list, columns_list):
    for idx, row in df.iterrows():
        if idx in index_list:
            for col in columns_list:
                if col in df.columns and row[col] != 0:
                    return True
    return False

def check_diagonal(df):
    for idx in df.index:
        if idx in df.columns and df.at[idx, idx] != 0:
            return True
    return False


def on_submit_button_click(button):
    global qgrid_supply_widget, qgrid_demand_widget, qgrid_transfer_widget, qgrid_start_widget, qgrid_end_widget, qgrid_matrix_widget
    global supply_names, demand_names, transfer_names, start_name, end_name
    global edited_supply_df, edited_demand_df, edited_transfer_df, edited_start_df, edited_end_df, edited_matrix_df
    global count_supply_nodes, count_demand_nodes, count_transfer_nodes
    
    with output:
        
        # Extract the edited values from the qgrid widgets
        edited_supply_df = qgrid_supply_widget.get_changed_df()
        edited_demand_df = qgrid_demand_widget.get_changed_df()
        edited_transfer_df = qgrid_transfer_widget.get_changed_df()
        edited_start_df = qgrid_start_widget.get_changed_df()
        edited_end_df = qgrid_end_widget.get_changed_df()
        edited_matrix_df = qgrid_matrix_widget.get_changed_df()
        
        if radio_buttons.value == 'Transshipment Problem':
            if check_dataframe(edited_matrix_df, supply_names, demand_names) or check_dataframe(edited_matrix_df, transfer_names, transfer_names):
                show_warning_message("Input Error! The value of a cell of transfer nodes should be None. Please check again.")
        
        if radio_buttons.value in ['Maximum Flow Problem', 'Traveling Salesperson Problem']:
            if check_diagonal(edited_matrix_df):
                show_warning_message("Input Error! The value of a cell in the diagonal should be None.")
                                     
        on_generate_all_constraints_button_click(button)

    button.disabled = True


def on_generate_all_constraints_button_click(button):
   with output:        
        # Handle 'General Model' radio button value
        if radio_buttons.value == 'General Model':
            general_model_generate_all_constraints()
            
        # Handle other radio button values except 'General Model'
        elif radio_buttons.value != 'General Model':
            network_model_generate_all_constraints()
            
        button.disabled = True

        
def general_model_generate_all_constraints():
    # The code corresponding to the 'General Model' radio button value
    # pass the value of input
    global qgrid_variables_widget, qgrid_constraints_widget
    global variable_name, n_variables, obj_coeff, obj, variable_type, cons_df, v_transfer_node    
    global qgrid_supply_widget, qgrid_demand_widget, qgrid_transfer_widget, qgrid_start_widget, qgrid_end_widget, qgrid_matrix_widget

 
    variables_df = qgrid_variables_widget.get_changed_df()
    variables_df = variables_df.loc[variables_df["Variable Name"] != '']

    matrix_constraints_df = qgrid_constraints_widget.get_changed_df()
    matrix_constraints_df = matrix_constraints_df[matrix_constraints_df['constraint type'].isin(['>', '<', '=', '>=', '<='])]

    variable_name = variables_df['Variable Name'].values.tolist()
    variable_name = [item for item in variable_name if item != '']            
    n_variables = len(variable_name)

    obj_coeff = variables_df['Objective coefficient'].values.tolist()

    # read objective
    obj = objective_dropdown.value
    if obj =='max':
        obj = -1
    elif obj =='min':
        obj = 1
    else:
        raise NameError('Objective cannot be recognized')


    #get variable type     

    variable_type = []
    v = variable_type_dropdown.value

    for c in range(n_variables):
        if not v:
            t = 'C' 
        elif v == 'cont':
            t = 'C'
        elif v == 'int':
            t = 'I'
        elif v == 'bin':
            t = 'B'
        else:
            raise NameError('Variable type cannot be recognized')
        variable_type.append(t)


    # get constraints
    constraint_columns = list(variables_df['Variable Name']) + ['constraint type', 'RHS values']

    # get constraints of non-neg
    range_value = range_dropdown.value
    values = []
    if range_value == 'non-negative':
        for i in range(n_variables):
            r_values = [0] * (n_variables + 2)
            r_values[i] = 1
            r_values[n_variables] = '>='
            r_values[n_variables + 1] = 0
            values.append(r_values)

    non_neg_df = pd.DataFrame(values, columns=constraint_columns)


    # get constraints of Lower Bound
    lower_bound_list = []
    for idx, row in variables_df.iterrows():
        if pd.notna(row['Variable Name']) and pd.notna(row['Variable Lower Bound']):
            new_row = [0] * n_variables
            new_row[idx-1] = 1
            new_row.extend(["<=", row['Variable Lower Bound']])
            lower_bound_list.append(new_row)
    lower_bound_df = pd.DataFrame(lower_bound_list, columns=constraint_columns)

    # get constraints of Upper Bound
    upper_bound_list = []
    for idx, row in variables_df.iterrows():
        if pd.notna(row['Variable Name']) and pd.notna(row['Variable Upper Bound']):
            new_row = [0] * n_variables
            new_row[idx-1] = 1
            new_row.extend(["<=", row['Variable Upper Bound']])
            upper_bound_list.append(new_row)
    upper_bound_df = pd.DataFrame(upper_bound_list, columns=constraint_columns)

    cons_df = pd.concat([matrix_constraints_df, lower_bound_df, upper_bound_df, non_neg_df], ignore_index=True)

    cons_df.columns = list(range(cons_df.shape[1]-2)) + cons_df.columns[-2:].tolist()

    for column in cons_df.columns:
        if column != 'constraint type':
            cons_df[column] = pd.to_numeric(cons_df[column])

    display(HTML("<hr>"))
    display(HTML('<h3 style="background-color: yellow;">STEP 3: All Constraints</h3>'))

    display(cons_df, run_gurobi_button)



def network_model_generate_all_constraints():
    # The code corresponding to other radio button values except 'General Model'
    # pass the value of input
    global qgrid_variables_widget, qgrid_constraints_widget
    global variable_name, n_variables, obj_coeff, obj, variable_type, cons_df    
    global qgrid_supply_widget, qgrid_demand_widget, qgrid_transfer_widget
    global qgrid_start_widget, qgrid_end_widget, qgrid_matrix_widget, v_transfer_node


    objective_value = objective_dropdown.value
    variable_type_value = variable_type_dropdown.value
    range_value = range_dropdown.value
    problem_type_value = problem_type_dropdown.value

    # Extract values from input boxes
    count_supply_nodes = box_count_supply_nodes.value
    count_demand_nodes = box_count_demand_nodes.value
    count_transfer_nodes = box_count_transfer_nodes.value

    # Extract the edited values from the qgrid widgets
    edited_supply_df = qgrid_supply_widget.get_changed_df()
    edited_demand_df = qgrid_demand_widget.get_changed_df()
    edited_transfer_df = qgrid_transfer_widget.get_changed_df()
    edited_matrix_df = qgrid_matrix_widget.get_changed_df()


    supply_names = qgrid_supply_widget.get_changed_df()['Name of Node'].values
    demand_names = qgrid_demand_widget.get_changed_df()['Name of Node'].values
    transfer_names = qgrid_transfer_widget.get_changed_df()['Name of Node'].values
    start_name = qgrid_start_widget.get_changed_df()['Name of Node'].values
    end_name = qgrid_end_widget.get_changed_df()['Name of Node'].values

    if radio_buttons.value == 'Maximum Flow Problem':
        edited_matrix_df.at[end_name, start_name] = 999999


    if radio_buttons.value == 'Traveling Salesperson Problem':
        supply_names = edited_matrix_df.index.tolist()
        demand_names = edited_matrix_df.index.tolist()

        edited_supply_df = pd.DataFrame({
            'Name of Node': supply_names,
            'Capacity of Node': [1]*len(edited_matrix_df)
        }, index=list(range(1, len(edited_matrix_df) + 1)))

        edited_demand_df = pd.DataFrame({
            'Name of Node': demand_names,
            'Demand of Node': [1]*len(edited_matrix_df)
        }, index=list(range(1, len(edited_matrix_df) + 1)))



    # get variable name
    n_column = edited_matrix_df.shape[1]
    n_row = edited_matrix_df.shape[0]
    n_variables = n_column * n_row
    row_name = edited_matrix_df.index.tolist()
    column_name = edited_matrix_df.columns.tolist() 
    variable_name = []
    for i in range(0,n_row):
        for j in range(0,n_column):
            v_name = row_name[i] + '_TO_' + column_name[j]
            variable_name.append(v_name)


    # get variable capacity
    v_supply = edited_supply_df['Capacity of Node'].astype(int).tolist()
    v_demand = edited_demand_df['Demand of Node'].astype(int).tolist()
    if count_transfer_nodes > 0 and radio_buttons.value != 'Traveling Salesperson Problem':    
        v_transfer = [None] * count_transfer_nodes
        v_supply = v_supply + v_transfer
        v_demand = v_transfer + v_demand


    # get variable types     
    variable_type = []
    for c in range(n_variables):
        if variable_type_value == 'cont':
            t = 'C'
        elif variable_type_value == 'int':
            t = 'I'
        elif variable_type_value == 'bin':
            t = 'B'
        else:
            raise NameError('Variable type cannot be recognized')
        variable_type.append(t)


    # get objective
    obj = objective_value
    if obj =='max':
        obj = -1
    elif obj =='min':
        obj = 1
    else:
        raise NameError('Objective cannot be recognized')


    # get obj coefficient  
    obj_coeff=[]
    if radio_buttons.value == 'Maximum Flow Problem':
        for i in range(n_variables):
            v = 0
            obj_coeff.append(v)
        obj_coeff[-1] = -1
    else:
        for i in range(0,n_row):
            for j in range(0,n_column):
                v = edited_matrix_df.iloc[i, j]
                if v == 0:
                    if obj == -1:
                        v = -99999999
                    else:
                        v = 99999999
                obj_coeff.append(v)


    ### get constraints

    # get constraints of supply
    values = []
    if len(supply_names) > 0:
        for i in range(n_row):
            r_values = [0] * (n_variables + 2)
            for j in range(n_column):
                r_values[i*n_column + j] = 1
            r_values[n_variables] = '<='
            r_values[n_variables + 1] = v_supply[i]
            if v_supply[i] is not None:
                values.append(r_values)
    constr_supply = values


    # get constraints of demand
    values = []
    if len(demand_names) > 0:
        for i in range(n_column):
            r_values = [0] * (n_variables + 2)
            for j in range(n_row):
                r_values[i+ j*n_column] = 1
            r_values[n_variables] = '>='
            r_values[n_variables + 1] = v_demand[i]
            if v_demand[i] is not None:
                values.append(r_values)
    constr_demand = values


    # get constraints of non-neg
    if range_value == 'non-negative':
        values = []
        for i in range(n_variables):
            r_values = [0] * (n_variables + 2)
            r_values[i] = 1
            r_values[n_variables] = '>='
            r_values[n_variables + 1] = 0
            values.append(r_values)
    constr_non_neg = values


    # get constraints of the capacity of the node
    # Extract all rows and all columns and convert to a flattened list
    node_capacity = edited_matrix_df.iloc[:n_row, :n_column].values.ravel().tolist()
    values = []
    for i in range(0,n_variables):
        r_values = [0] * (n_variables + 2)
        r_values[i] = 1
        r_values[n_variables] = '<='
        r_values[n_variables + 1] = node_capacity[i]
        values.append(r_values)
    constr_capacity = values


    # get constraints of in_n_out
    if radio_buttons.value != 'Maximum Flow Problem':
        transfer_names = edited_transfer_df['Name of Node'].tolist()
    else:
        transfer_names = edited_matrix_df.index.tolist()

    v_transfer_node = []
    for i in range(len(transfer_names)):
        transfer_node = transfer_names[i]
        transfer_node_row = row_name.index(transfer_node)
        transfer_node_column = column_name.index(transfer_node)
        v_transfer_node.append([transfer_node, [transfer_node_column, transfer_node_row]])

    values = []
    for i in range(len(transfer_names)):
        r_values = [0] * (n_variables + 2)
        transfer_node_column = v_transfer_node[i][1][0]
        transfer_node_row = v_transfer_node[i][1][1]
        for j in range(n_row):
            r_values[j*n_column + transfer_node_column] = 1
            r_values[transfer_node_row*n_column + j] = -1
        r_values[n_variables] = '='
        r_values[n_variables + 1] = 0
        values.append(r_values)
    constr_in_n_out = values


    ## read constraints as dataframe
    if radio_buttons.value == 'Maximum Flow Problem':
        cons = constr_in_n_out + constr_capacity
    else:
        cons = constr_in_n_out + constr_supply  + constr_demand

    if range_value == 'non-negative':
        cons+= constr_non_neg

    cons_df = pd.DataFrame()
    cons_df = pd.DataFrame(cons)
    cons_df.rename(columns={n_variables:'constraint type', 
                            n_variables+1:'RHS values'}, inplace=True)


    cons_df['RHS values'] = pd.to_numeric(cons_df['RHS values'], errors='coerce')

    display(HTML("<hr>"))
    display(HTML('<h3 style="background-color: yellow;">STEP 5: All Constraints</h3>'))

    display(cons_df, run_gurobi_button)
    
    
    
def on_run_gurobi_button_click(button):
    
    global cons_df

    # Set up the Gurobi model
    m = Model()

    # Add variables to the model
    set_up_variables(m)

    # Set up the objective of the model
    set_up_objective(m)

    # Add constraints to the model
    add_constraints_to_model(m, cons_df)

    # Optimize the model
    m.optimize()

    # Handle the 'Traveling Salesperson Problem' scenario separately
    if radio_buttons.value == 'Traveling Salesperson Problem':
        handle_tsp_problem(m)
    else:
        # Display results
        display_final_results(m)


def set_up_variables(m):
    # Set up the variables for the Gurobi model
    global variable_name, n_variables, variable_type
    global x
    
    x = m.addVars(n_variables)
    for i in range(n_variables):
        x[i].setAttr('VarNAME', variable_name[i])
        x[i].setAttr('vType', variable_type[i])


def set_up_objective(m):
    # Set up the objective for the Gurobi model
    global obj_coeff, obj, n_variables    
    global x

    objective = quicksum(obj_coeff[i] * x[i] for i in range(n_variables))
    m.setObjective(objective, obj)


def add_constraints_to_model(m, cons_df):
    # Add constraints to the Gurobi model
    global n_variables
    global x

    if cons_df.shape[0]:
        ## add le constraints
        le_const = cons_df[cons_df['constraint type'] == '<=']
        le_const.reset_index(inplace=True, drop=True)
        for i in range(le_const.shape[0]):
            m.addConstr(quicksum(le_const.iloc[i,j]* x[j] for j in range(n_variables)) <= le_const['RHS values'][i])
        ## add ge constraints
        ge_const = cons_df[cons_df['constraint type'] == '>=']
        ge_const.reset_index(inplace=True, drop=True)
        for i in range(ge_const.shape[0]):
            m.addConstr(quicksum(ge_const.iloc[i,j]* x[j] for j in range(n_variables)) >= ge_const['RHS values'][i])
        ## add eq constraints
        eq_const = cons_df[cons_df['constraint type'] == '=']
        eq_const.reset_index(inplace=True, drop=True)
        for i in range(eq_const.shape[0]):
            m.addConstr(quicksum(eq_const.iloc[i,j]* x[j] for j in range(n_variables)) == eq_const['RHS values'][i])


def handle_tsp_problem(m):
    # Handle the 'Traveling Salesperson Problem' scenario
    # transfer the result to list

    global variable_name, n_variables, obj_coeff, obj, variable_type, cons_df, v_transfer_node    
    global x

    result_list = m.getVars()
    result_list_1 = [] 

    for i in range(len(result_list)):
        if (result_list[i].x > 0.99):
            temp = [i, result_list[i].varName]
            result_list_1.append(temp)

    for i in range(len(result_list_1)):
        s = result_list_1[i][1].split('_TO_',1)
        result_list_1[i][1] = s


    # count the numbers of the nodes of subtour

    n_result = len(result_list_1)

    subtour = []
    subtour.append(result_list_1[0])

    while subtour[-1][1][1] != subtour[0][1][0]:
        for i in range(n_result):
            if result_list_1[i][1][0] == subtour[-1][1][1]:
                subtour.append(result_list_1[i])

    n_subtour = len(subtour)

    subtour_index = []

    for i in subtour:
        subtour_index.append(i[0])


    # iteration the result

    while n_subtour != n_result:

        # add new constrain
        values = []
        constr_in_n_out_update = []

        n_v_transfer_node = len(v_transfer_node)

        for i in range(n_v_transfer_node):
            values = [0] * (n_variables + 2)

        for i in subtour_index:
            values[i] = 1

        values[n_variables] = '<='
        values[n_variables + 1] = n_subtour - 1

        constr_in_n_out_update = values

        cons_df.loc[len(cons_df.index)] = constr_in_n_out_update

        print(cons_df)

        m = Model()

        # add variables
        x=m.addVars(n_variables)
        # set types, lb, ub of variables
        for i in range(n_variables):
            x[i].setAttr('VarNAME', variable_name[i])
            x[i].setAttr('vType', variable_type[i])

        # set objective
        objective = quicksum(obj_coeff[i] * x[i] for i in range(n_variables))
        m.setObjective(objective, obj)

        # new constraints
        if cons_df.shape[0]:
            ## add le constraints
            le_const = cons_df[cons_df['constraint type'] == '<=']
            le_const.reset_index(inplace=True, drop=True)
            for i in range(le_const.shape[0]):
                m.addConstr(quicksum(le_const.iloc[i,j]* x[j] for j in range(n_variables)) <= le_const['RHS values'][i])
            ## add ge constraints
            ge_const = cons_df[cons_df['constraint type'] == '>=']
            ge_const.reset_index(inplace=True, drop=True)
            for i in range(ge_const.shape[0]):
                m.addConstr(quicksum(ge_const.iloc[i,j]* x[j] for j in range(n_variables)) >= ge_const['RHS values'][i])
            ## add eq constraints
            eq_const = cons_df[cons_df['constraint type'] == '=']
            eq_const.reset_index(inplace=True, drop=True)
            for i in range(eq_const.shape[0]):
                m.addConstr(quicksum(eq_const.iloc[i,j]* x[j] for j in range(n_variables)) == eq_const['RHS values'][i])

        ## run the model
        m.optimize()

        # print optimal decisions (if not listed, that means their optimal value = 0)
        m.printAttr('X')


        # transfer the result to list
        result_list = m.getVars()
        result_list_1 = [] 

        for i in range(len(result_list)):
            if (result_list[i].x > 0.99):
                temp = [i, result_list[i].varName]
                result_list_1.append(temp)

        for i in range(len(result_list_1)):
            s = result_list_1[i][1].split('_TO_',1)
            result_list_1[i][1] = s

        # count the numbers of the nodes of subtour
        n_result = len(result_list_1)

        subtour = []
        subtour.append(result_list_1[0])

        while subtour[-1][1][1] != subtour[0][1][0]:
            for i in range(n_result):
                if result_list_1[i][1][0] == subtour[-1][1][1]:
                    subtour.append(result_list_1[i])

        n_subtour = len(subtour)

        subtour_index = []

        for i in subtour:
            subtour_index.append(i[0])

        subtour_index
        
    display_final_results(m)

def display_final_results(m):
    # Display the final results
    display(HTML("<hr>"))
    display(HTML('<h3 style="background-color: yellow;">FINAL RESULT</h3>'))
    
    # print optimal objective value
    m.objVal
    print(m.objVal)

    # print optimal decisions (if not listed, that means their optimal value = 0)
    m.printAttr('X')
    


# Buttons
def create_styled_button(description, button_color='blue', font_color='white'):
    button = widgets.Button(
        description=description,
        layout=widgets.Layout(width='150px', height='auto'),
        style=widgets.ButtonStyle(button_color=button_color)
    )
    # Add the custom CSS class to the button
    button.add_class('custom-button-style')
    return button

# Create buttons and link them to their respective handler functions
buttons_descriptions_funcs = [
    ("Next", on_first_next_button_click),
    ("Next", on_generate_tables_button_click),
    ("Next", on_generate_matrix_button_click),
    ("Next", on_submit_button_click),
    ("Next", on_input_constraints_button_click),
    ("Next", on_generate_all_constraints_button_click),
    ("Run Gurobi", on_run_gurobi_button_click)
]

buttons = []
for desc, func in buttons_descriptions_funcs:
    button = create_styled_button(desc)
    button.on_click(func)
    buttons.append(button)

# Button names
first_next_button, generate_tables_button, generate_matrix_button, submit_button, input_constraints_button, generate_all_constraints_button, run_gurobi_button = buttons

# Set CSS
def apply_custom_css():
    """
    Applies custom CSS styles for qgrid and output display.
    """
    custom_css = """
    <style>
        .q-grid .slick-cell {
            text-align: center;
        }
        .q-grid .slick-header-column.ui-state-default {
            text-align: center;
            display: flex;
            justify-content: center;
            align-items: center;
        }
        .output_scroll { 
            height: auto !important; 
        }
        .custom-button-style {
            color: white;
        }
    </style>
    """
    display(HTML(custom_css))

apply_custom_css()

# Display the initial interface
display(HTML("<hr>"))
display(HTML('<h3 style="background-color: yellow;">STEP 1: Select Problem Category</h3>'))
display(radio_buttons, output)


Set parameter Username

--------------------------------------------
--------------------------------------------

Academic license - for non-commercial use only - expires 2023-09-15


RadioButtons(description='Category:', options=('General Model', 'Transportation Problem', 'Transshipment Probl…

Output()